In [1]:
import os
import pandas as pd
import numpy as np

def find_primary_secondary(co_matrix):
    primary_sum = 0
    primary_node = 0
    secondary_nodes = []

    for i in range(20):
        sum_correlation = np.sum(co_matrix[i])

        if sum_correlation > primary_sum:
            primary_sum = sum_correlation
            primary_node = i

    for i in range(20):
        if co_matrix[primary_node][i] >= 0.3 and primary_node != i:
            secondary_nodes.append(i)

    return primary_node, secondary_nodes

def delete_secondary_primary(co_matrix, primary_node, secondary_nodes):
    for i in range(20):
        co_matrix[primary_node][i] = 0
        co_matrix[i][primary_node] = 0
        for ele in secondary_nodes:
            co_matrix[ele][i] = 0
            co_matrix[i][ele] = 0
    return co_matrix

In [2]:
# Initialize a list to store the results
results = []

for k in range(100):
    if k == 50:
        continue

    data = pd.read_csv(f"D:\jupyter_PY\subcluster100\cluster_{k.0}data.csv")  
    correlation_matrix = np.zeros((20, 20))

    for i in range(20):
        for j in range(20):
            cluster_i_values = data[data['Cluster_Label'] == i]['Serving RSSI_4G'].values
            cluster_j_values = data[data['Cluster_Label'] == j]['Serving RSSI_4G'].values

            min_len = min(len(cluster_i_values), len(cluster_j_values))
            if min_len > 0:
                cluster_i_values = cluster_i_values[:min_len]
                cluster_j_values = cluster_j_values[:min_len]

                correlation_matrix[i, j] = np.corrcoef(cluster_i_values, cluster_j_values)[0, 1]
            else:
                correlation_matrix[i, j] = 0

    correlation_matrix = np.nan_to_num(correlation_matrix, nan=0.0)

    print(f"\nFor matrix {k}:\n")
    c = 0
    while c < 2:
        p_node, s_node = find_primary_secondary(correlation_matrix)
        if any(item == 0 for item in s_node):
            c += 1
        if p_node == 0:
            c += 1
        if c == 2:
            break
        correlation_matrix = delete_secondary_primary(correlation_matrix, p_node, s_node)
        print('Primary Node:', p_node)
        if s_node:
            print('Secondary Node:', s_node)

        # Calculate the median of 'Longitude', 'Latitude', 'Serving RSSI_4G' for the primary node
        primary_data = data[data['Cluster_Label'] == p_node]
        median_longitude = primary_data['Longitude'].median()
        median_latitude = primary_data['Latitude'].median()
        median_rssi = primary_data['Serving RSSI_4G'].median()

        # Append the results to the list
        results.append({
            'slNo': len(results) + 1,
            'Cluster': k,
            'Primary Node': p_node,
            'Median Longitude': median_longitude,
            'Median Latitude': median_latitude,
            'Median RSSI': median_rssi
        })


SyntaxError: f-string: invalid syntax. Perhaps you forgot a comma? (2632954540.py, line 8)